In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import requests

In [2]:
import os
import random
import time

from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import (
    MoveTargetOutOfBoundsException,
    TimeoutException,
    WebDriverException,
)

import chromedriver_binary

In [3]:
os.makedirs('data/', exist_ok=True)

In [4]:
def open_browser():
    """
    Opens a new automated browser window with all tell-tales of automated browser disabled
    """
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")
    
    # remove all signs of this being an automated browser
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)

    # open the browser with the new options
    driver = webdriver.Chrome(options=options)
    return driver

In [5]:
driver = open_browser()

In [6]:
# setting up the url as one allowing to enter straight onto the politcs forum of the website
url = 'https://www.fmkorea.com/politics'
driver.get(url)

In [7]:
search_box = driver.find_element(
    By.XPATH, 
    './/input[@name="search_keyword"]'
)

In [8]:
search_term = '짱깨'
search_box.send_keys(search_term)

In [9]:
def press_enter(driver):
    """
    Sends the ENTER to a webdriver instance.
    """
    actions = ActionChains(driver)
    actions.send_keys(Keys.ENTER)
    actions.perform()

In [10]:
press_enter(driver)

In [11]:
def get_results_on_page(driver, fn_out):
    """
    Scrolls to load all listings and then saves them to `fn_out`.
    If you found a better approach, replace this function
    """
    N = 0
    while True:
        # get all the listings, and scroll to the last one, then wait two seconds.
        posts = driver.find_elements(By.XPATH, './/span[@class="time"]')
        last_post = posts[-1]

        # you can use selenium to issue JavaScript commands:
        driver.execute_script("arguments[0].scrollIntoView();", last_post)
        N_posts = len(posts)
        if N_posts == N:
            break
        N = N_posts
        time.sleep(2)
        
    # how to save what the emulator sees
    with open(fn_out, 'w') as f:
        f.write(driver.page_source)

In [12]:
# my dataset would range between the year 2022 - 2016
# the codes below taking onto the page on which 2022 posts appears, which is page 31
# and finding the exact page with the last 2022 post manually...
# 'next page' button is supposed to turn 10 pages, not a single one
# page 24 is the page on which the last post in 2022 appears
# page 24 will be the start point

while True:
    posts = driver.find_elements(By.XPATH, './/td[@class="time"]')
    last_post = posts[-1]
    driver.execute_script("arguments[0].scrollIntoView();", last_post)
    next_page = driver.find_element(By.XPATH, './/a[text() = "다음 "]')

    if '2022' in last_post.text.split('.') :
        break

    # click the "next page" link
    next_page.click()
    time.sleep(2)

In [13]:
# finding the end point
# page 376 is the last page on which the last post of 2016 appears

while True:
    posts = driver.find_elements(By.XPATH, './/td[@class="time"]')
    last_post = posts[-1]
    driver.execute_script("arguments[0].scrollIntoView();", last_post)
    next_page = driver.find_element(By.XPATH, './/a[text() = "다음 "]')

    if '2015' in last_post.text.split('.') :
        break

    # click the "next page" link
    next_page.click()
    time.sleep(2)

KeyboardInterrupt: 

In [53]:
# saving the current page 24

page_source = driver.page_source

# Save the page source to a file
with open('page_source.html', 'w', encoding='utf-8') as file:
    file.write(page_source)

In [35]:
f'.//a[@href="/index.php?mid=politics&amp;search_keyword=%EC%A7%B1%EA%B9%A8&amp;search_target=title_content&amp;page={page_n}"]'

'.//a[@href="/index.php?mid=politics&amp;search_keyword=%EC%A7%B1%EA%B9%A8&amp;search_target=title_content&amp;page=24"]'

In [14]:
# saving htmls btween page 25 and 376

page_n = 24
while True: 
    base_url="/index.php?mid=politics&amp;search_keyword=%EC%A7%B1%EA%B9%A8&amp;search_target=title_content&amp;page="
    fn_out = f'data/fmkorea_짱깨_page_{page_n}.html'
    get_results_on_page(driver, fn_out)
    page_n += 1

    # stop after 376
    if page_n == 376:
        break

    # click the "next page" link
    next_page = driver.find_element(By.XPATH, 
                                    f'.//a[contains(@href, "/index.php?mid=politics") and contains(@href, "page={page_n}")]')
    next_page.click()
    time.sleep(2)

IndexError: list index out of range

In [58]:
# soup.select